In [3]:
# prepare for simulation

# user-settable params for the sim
numstudents = 200

MaxTaskGroupSize = 25
MinTaskGroupSize = 12
TaskGroupCount = 8 # not including the task group that is pup-only
deeppredcount = 5 # not used in sim
maxsk = 21 #not used in sim
numskills = 30 
numtasks = 6
numSkillsonPL = 8
maxPLsPerStud = 20
maxSkillsPerPL = 8 # not used in sim
uniquetasks = [i for i in range(numtasks)]
uniqueskills = [i for i in range(numskills)]
uniquestudents = [i for i in range(1,numstudents+1)]


import inspect
import numpy as np
import pandas as pd

# function Parameter().genstr() generates the appropriate string of parameter data for playlist-specific scores
class Parameter:
    def __init__(self, strParameter):
        self.strParameter = strParameter
        self.studlist = []
    def genstr(self):
        str1 = ' '
        # cases. all generate matrix (2-d array) of uniform random numbers
        if self.strParameter=='IndividualPlaylistQualityScore':
            randomarray = np.random.rand(numstudents, maxPLsPerStud)
        elif self.strParameter=='PlaylistQualityForTask':
            randomarray = np.random.rand(numstudents, maxPLsPerStud)
        elif self.strParameter=='IsValidForTaskGroup':
            randomarray = np.random.binomial(1,0.8, size=(numstudents, maxPLsPerStud))
        elif self.strParameter=='TaskPredecessorCount':
            randomarray = np.random.random_integers(0,5, size=(numstudents, maxPLsPerStud))
        elif self.strParameter=='TaskPredecessorCount':
            randomarray = np.random.random_integers(0,5, size=(numstudents, maxPLsPerStud))
        elif self.strParameter=='rollover':
            randomarray = np.random.rand(numstudents, maxPLsPerStud)/2.2 # uniform rannge in 0 to 0.45
        elif self.strParameter=='listtasks':
            randomarray = np.random.random_integers(0,numtasks, size=(numstudents, maxPLsPerStud)) 
        
        # massage into string: numstudent lines of numPL entries     
        for i, elem in enumerate(uniquestudents): #iter on jsut 1 and 2 in addition to 0 and 3
            studentname= elem
            str1 = str1 + '\n '   + str(studentname) + '  ' + ' '.join(str(randomarray[i][p]) for p in range(maxPLsPerStud) ) #[sk][i][p]

        if self.strParameter=='listtasks':
            self.strParameter = 'playlist_p_for_i_istask'  #playlist is task what is named differently 
        
        return '\nparam  '  +self.strParameter+ ' : '  +  ' '.join(str(p) for p in range(maxPLsPerStud)) + ' :=  ' + str1  + ' ;'

In [5]:
# SIMULATION
# strings near the base of this cell lists the data generated, split into sets and the data/scores

from random import randint, randrange, random, uniform, normalvariate, shuffle, seed
import numpy, numpy.random as rand
from datetime import datetime
from operator import div


seed(12345)

strTG = 'set TASKGROUP := '+ ' '.join(str(i) for i in range(TaskGroupCount+1)) + ' ;' #bc don't include TG0 in the count
strMinMaxTGsize = 'param MAXTASKGROUPSIZE := '+ str(MaxTaskGroupSize) + ' ; \nparam MINTASKGROUPSIZE := '+ str(MinTaskGroupSize)  + ' ;'

## Generate data for playlist-specific scores
strIPQSscore = Parameter('IndividualPlaylistQualityScore').genstr()
strPQFTscore = Parameter('PlaylistQualityForTask').genstr()
strIsValidscore = Parameter('IsValidForTaskGroup').genstr()
strTaskPred = Parameter('TaskPredecessorCount').genstr()
#strExpTag = Parameter('PlaylistExperienceTag').genstr()
strRollover = Parameter('rollover').genstr()

# gen data for NumSkillsonstudentPL and strNumSkillsonPL : this is a weird way to do things
def makeStringFromTuple(tuple1):
    return ' '+ str(tuple1[0])+ ' '+str(tuple1[1])+ '  '
zipped= zip(uniquestudents,[numSkillsonPL for i in range(numstudents)])
strNumSkillsonstudentPL = ''.join(makeStringFromTuple(i) for i in zipped)
strNumSkillsonstudentPL = 'param: STUDENT : numSkillsonstudentPL := ' + strNumSkillsonstudentPL + ' ;'
strNumSkillsonPL = 'param numSkillsonPL:=  ' + str(numSkillsonPL) + ' ;' 

# TASK that is attached to each playlist
strAllTasks = Parameter('listtasks').genstr()

## TASKS : the set of tasks
uniquetasks.sort()
strSetTasks = 'set TASK := '+' '.join(str(i) for i in uniquetasks) + ' ;'


## SKILLS : the set of skills
uniqueskills.sort()
strSetSkills = 'set SKILL :=  '+' '.join(str(j+1) for j in uniqueskills) + '  ;' # starts 1. zero is reserved for no nth skill

## PLAYLIST indexes (ie up to maxPLsPerStud)
strPLcommonIndex = 'set PLCOMMONINDEX := ' + ' '.join(str(p) for p in range(maxPLsPerStud)) + ' ;'

### end SETs

## generate the skills that comprise each PLAYLIST
allplaylists = []
for i,elem in enumerate(uniquestudents):
    studentplaylists = []
    for j in range(maxPLsPerStud):
        playlist=[j for j in range(numskills)]
        np.random.shuffle(playlist)
        playlist = playlist[0:numSkillsonPL]
        playlist.sort(reverse=True)  # sort descending
        studentplaylists.append(playlist)
    allplaylists.append(studentplaylists)

    ## then convert playlists to string structure
# by going first from  list of playlistskills indexed by student then playlist then index ie allplaylists[student][playlistindex][skillindex]
# to list of skills by  skillindex then student then playlist ie elemPL[skillindex][student][playlistindex]
elemPL = []
#for each sk
for sk in range(maxSkillsPerPL):
    listgivenindex = []
    # for each row i(student i)
    for i in range(len(allplaylists)): #for numstudents
        studlist = []
        for p in range(maxPLsPerStud): #for numplaylists, wheter exist or not
            try:
                studlist.append(allplaylists[i][p][sk])
            except IndexError:
                studlist.append(0)
        listgivenindex.append(studlist)
    elemPL.append(listgivenindex)

# stringify elemPL
strAllSkills = ''
for sk in range(maxSkillsPerPL):
    str1 = ' '
    for i, elem in enumerate(uniquestudents): #iter on jsut 1 and 2 in addition to 0 and 3
        studentname= elem
        str1 = str1 + '\n '   + str(studentname) + '  ' + ' '.join(str(elemPL[sk][i][p]) for p in range(maxPLsPerStud))
        #param  sk  : 1 2 3 4 :=   followed by many rows of students
    strAllSkills =  strAllSkills + '\nparam  sk' + str(sk+1) + ' : '  +  ' '.join(str(p) for p in range(maxPLsPerStud)) + ' :=  ' + str1  + ' ;'

# iterable strings to print
strings = [strTG, strMinMaxTGsize, strSetSkills, strSetTasks, strPLcommonIndex, strNumSkillsonstudentPL, strNumSkillsonPL,
           strAllSkills, strAllTasks, 
           strRollover, strIPQSscore , strPQFTscore , strIsValidscore, strTaskPred] # excluded strExpTag

# set filename, and write the file
fname = 'TGAdata' + str(numstudents) + 'x'+str(numtasks) +'tskx'+str(maxPLsPerStud) +'PLsx'+str(maxSkillsPerPL) +'varPLskills' +' '+ datetime.now().strftime("%Y-%m-%d %H:%M")+ '.txt' #+str(numskillsperPL)+'perPLmenuskills'
fname = fname.replace(':','', 3)
with open(fname, 'w') as f: # Open 'newfile.txt' for writing  # Open 'newfile.txt' for writing
    for string in strings:
        f.write(string)           # Here '\n' means new line
        f.write('\n \n')

In [6]:
# generate xml file with name xmlfname,  based on the below 3 files

modelFilename = "chanan_aggpltaskNoPup.txt"
dataFilename = fname
commandsFilename = "commandsOps.txt"

xmlfname = 'xmlsimfile200.xml' 
# fname = 'xmlfile' + datetime.now().strftime("%Y-%m-%d %H:%M")+ '.xml'
# fname = fname.replace(':','', 3)


import os as os
def filetostring(strFilename,buffersize):
    fd = os.open(strFilename, os.O_RDWR)
    string = os.read(fd, buffersize)
    os.close(fd)
    return string
def filestoxml():
    buffersizemodelfile = 20000
    buffersizedatafile = 2500000
    buffersizecommandsfile = 10000
    
    with open(xmlfname, 'w') as f:
        strXMLstart = ('<document> \n <category>milp</category> \n <solver>CPLEX</solver> \n <inputMethod>AMPL</inputMethod> \n ' +
 '<model><![CDATA[ \n' + filetostring(modelFilename, buffersizemodelfile) + '\n ]]></model> '  ) # end XMLstart
        f.write(strXMLstart)
        f.write('\n \n') 
    
        strXMLdata = ('<data><![CDATA[ \n'  + filetostring(dataFilename,buffersizedatafile)  + '\n ]]></data> ' )
        f.write(strXMLdata)
        f.write('\n \n') 
        
        strXMLend = ('<commands><![CDATA[ \n' + filetostring(commandsFilename,buffersizecommandsfile)  + '\n ]]></commands>  ' )
        f.write(strXMLend)
        f.write('\n \n') 
        
        
        strXMLemail = ('<email> dlau@newclassrooms.org </email> \n </document> ' )
        f.write(strXMLemail)
        f.write('\n \n') 
    return 0
#
filestoxml()

# inspect.getmembers(os.pipe)

OSError: [Errno 2] No such file or directory: 'chanan_aggpltaskNoPup.txt'

In [ ]:
%run NeosClient.py      FILENAME.xml
# run the xml file that was generated above
# default setting in commandsop.txt is for 5minutes, which can only do 20 students

In [ ]:
%run NeosClient.py xmlsimfile200.xml
# 600 couldn do in 10min; feas only (noobj objno=0) can't do in 3600 , can't do iis ,  1200 ,no iis mintaskgroupsize 1

JobNumber = 3970817 
Job 3970817 dispatched
password: cSGwDgqj
---------- Begin Solver Output -----------
Job submitted to NEOS HTCondor pool.


In [ ]:
%run NeosClient.py xmlsimfile200.xml
# 600 couldn do in 10min; feas only (noobj objno=0) can't do in 3600 ,  7200 to test

JobNumber = 3970818 
Job 3970818 dispatched
password: dgNWYpuo
---------- Begin Solver Output -----------
Job submitted to NEOS HTCondor pool.


In [ ]:
%run NeosClient.py xmlsimfile80.xml
# 1200

JobNumber = 3959738 
Job 3959738 dispatched
password: tkNSplmr
---------- Begin Solver Output -----------


In [ ]:
%run NeosClient.py xmlsimfile80.xml
# 3600 sec, and the put feas option fpheur=2

JobNumber = 3970419 
Job 3970419 dispatched
password: UWZaLNzi


In [ ]:
%run NeosClient.py xmlsimfile80.xml
# 1200 sec, and the put feas option fpheur=2, 1 = each integer feasible solution 3970510 bobund = 0.2

In [ ]:
%run NeosClient.py xmlsimfile80.xml
# 1200 sec, and feas fpheur=-1 , bound=0.05

JobNumber = 3970438 
Job 3970438 dispatched
password: AvtHqiLQ


In [ ]:
%run NeosClient.py xmlsimfile80.xml
# 1200 sec, and feas fpheur=1 , bound=0.20

JobNumber = 3970439 
Job 3970439 dispatched
password: cksHKUoX
